In [52]:
import numpy as np
import pandas as pd

In [53]:
book_df = pd.read_csv('data/Books.csv')
ratings_df = pd.read_csv('data/Ratings.csv')
user_df = pd.read_csv('data/Users.csv')
ratings_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [54]:
ratings_df = ratings_df[ratings_df['Book-Rating'] > 0]
ratings_df = ratings_df[ratings_df['User-ID'].isin(ratings_df['User-ID'].unique()[0:8000])]
ratings_df.shape

(50139, 5)

In [55]:
book_user_rating = book_df.merge(ratings_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)
book_user_rating.head()

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,8,5
1,0002005018,Clara Callan,Richard Bruce Wright,11676,8
2,0060973129,Decision in Normandy,Carlo D'Este,2954,8
3,0399135782,The Kitchen God's Wife,Amy Tan,11676,9
4,0671870432,PLEADING GUILTY,Scott Turow,11676,8


In [56]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [57]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-Rating').fillna(0)

In [59]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,30030,30031,30032,30033,30034,30035,30036,30037,30038,30039
User-ID,,,,,,,,,,,,,,,,,,,,,
8,5.0,0.0,0.0,0.0,5.0,5.0,5.0,6.0,6.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
